## Preliminaries

In [ ]:
import numpy as np
import cv2
import math
import copy
import matplotlib.pyplot as plt

def plot_1d(data, xlabel = "timestamp", ylabel = ""):
    plt.figure(figsize=(7, 4))
    plt.plot(data)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

from matplotlib.animation import FuncAnimation
import math
from matplotlib import rc
rc('animation', html='jshtml')

def animate(trajectory, axes_limits=[-5, 5, -5, 5], frames = 50, step = 1):
    x_hist_, y_hist_, _ = trajectory

    x_hist = x_hist_[::step]
    y_hist = y_hist_[::step]

    plt.ioff()

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.axis(axes_limits)
    ax.set_aspect("equal")

    point1, = ax.plot(0,1, marker="o", label="location")
    ax.legend()
    ax.grid()
    ax.set_xlabel("$x$")

    def update(t):
        x = x_hist[int(t) % len(x_hist)]

        point1.set_data([x],[0])

        return point1,

    ani = FuncAnimation(fig, update, interval=1000/24, blit=True, repeat=True,
                    frames=frames)
    plt.ion()

    return ani

# Linear Quadratic Regulator

## Context

Let us consider a linear system $\dot{x} = A x + B u$

During today's seminar we will focus on a way to synthesize full-state feedback $u = - K x$ for this system that will minimize quadratic cost in the form of $\int_{0}^{\infty} x^T Q x + u^T R u \, dt$.

The properties of linear systems allow for the derivation of the controllers of such type, which is extremely handy in cases when the onboard computational resources are limited and at the same time some kind of optimality is required. In contrast to PID controllers, that could require manual tuning, there is a closed form for the LQ controller that stabilizes the system and (informally speaking) keeps the energy consumption of the controller reasonable.

MPC on the other hand can handle extremely difficult problems, such as non-convex state and control limitations and a wide range of non-linear objectives, but it comes with the cost of finite planning horizon and heavy computational burden on the robot's computer.

However, it should be noticed that LQR and MPC are neither polar opposites, nor two incompatible approaches. Discrete finite-horizon LQR is essentially linear MPC.

## Problem statement and assumptions

Let us express the desired controller in the closed form, reducing a problem to number of matrix multiplications.

First, let us state the problem in terms of the requirements to all the matrices and provide some explanation to their meaning.

We will consider a system with the state dynamics of $\dot{x} = A x + B u$, where $x$ is n-dimensional and $u$ is m-dimensional. Please keep the shapes of all the matrices in mind during the derivation.

The problem of stabilizing the system, i.e. bringing the state $x$ to zero, will be considered.

Along the way we are going to minimize the cost functional $J$ that is given by $\int_{0}^{\infty} x^T Q x + u^T R u \, dt$. Why such a form was chosen? First, this quadratic cost will be equal to zero if the deviation from the desired state is absent and the control effort is zero as well. Second, the properties of the quadratic functions allow for a handful of useful of minimization techniques.

Let us closely examine the cost finctional. We will assume the matrix $Q$ to be symmetric positive semi-definite, meaning $x^T Q x \geq 0$ for any $x$. It makes sense to do so, because otherwise there will be states that deviate from the desired and at the same time decrease the functional that we consider to be a "cost" of the state-action trajectory in the episode.

Second, we will assume matrix $R$ to be positive-definite. Otherwise there will be some non-zero control outputs that do not cost anything or even have negative cost, which is on the conceptual level equivalent to some speed and steering of the car that result in the tank filling up with fuel.

## Derivation

Now let us add a term to the cost function and instantly subtract it, which will not change its value. We require the matrix P to be symmetrical here, although it is not known yet.

$J = \int_{0}^{\infty} [x^T Q x + u^T R u \,] \ dt = x_0^T P x_0 - x_0^T P x_0 + \int_{0}^{\infty} [x^T Q x + u^T R u \,] dt = $

Let us move $- x_0^T P x_0$ under the integral:

$= x_0^T P x_0 + \int_{0}^{\infty} [\frac{d}{dt} x^T P x + x^T Q x + u^T R u \,] dt = $

Then let us expand the first term under the integral with the product rule:

$= x_0^T P x_0 + \int_{0}^{\infty} ]\dot{x}^T P x + x^T P \dot{x} + x^T Q x + u^T R u \,] dt = $

Now let us substitute $\dot{x}$ with the right hand-side of the system dynamics:

$= x_0^T P x_0 + \int_{0}^{\infty} ](A x + B u)^T P x + x^T P (A x + B u) + x^T Q x + u^T R u \,] dt = $

After that a couple of square completions gives

$= x_0^T P x_0 + \int_{0}^{\infty} [x^T (A^T P + P A + Q - P B R^{-1} B^T P) x + (u + R^{-1} B^T P x)^T R (u + R^{-1} B^T P x) \,] dt$

Now let us carefully examine this result.

First, there are two terms here: the first one and the integral. Let us recall that **the overall value of the cost does not depend on the choice of matrix $P$**.

The matrix $P$ can be chosen such that it satisfies the Algebraic Riccati Equation $A^T P + P A + Q - P B R^{-1} B^T P = 0$.

The cost then takes form of

$x_0^T P x_0 + \int_{0}^{\infty} [(u + R^{-1} B^T P x)^T R (u + R^{-1} B^T P x) \,] dt$

Since R is positive definite and since the first term does not depend on the control $u$, the cost will be minimized if the control is precisely $u = - R^{-1} B^T P x$, which is exactly the full state feedback that we are looking for ($K = R^{-1} B^T P$).

Equation $A^T P + P A + Q - P B R^{-1} B^T P = 0_{n \times n}$ is a matrix Algebraic Riccati Equation (ARE). It becomes rather sophisticated to solve it by hand even for relatively small systems. Luckily enough, there are software tools that solve them automatically.

With this in mind, let us consider a specific system with simple linear dynamics. The plan for the rest is the following:
- a system with 1 degree of freedom (point with mass) will be considered
- simple PD feedback will be implemented and assessed in terms of the cost of the state-action trajectory
- LQR (continious and infinite-time!) will be applied to the same problem (which is discrete and finite-time)
- solving Algebraic Riccati Equations with scipy will be considered
- LQR will be applied to more complex systems

## Simulating a system

Let us consider a system with a mass moving horizontally without friction along one axis.

$x = \begin{pmatrix}
p \\
\dot{p}
\end{pmatrix}
$, where $p$ stands for position

Let us control this system by exerting force onto it.

$u = \begin{pmatrix}
F
\end{pmatrix}
$

The continuous system dynamics in such a case will take the form of

$\dot{x} = \begin{pmatrix}
\dot{p} \\
\ddot{p}
\end{pmatrix}$
$ = A x + B u
 =$
$ \begin{pmatrix}
0 & 1 \\
0 & 0
\end{pmatrix}
\begin{pmatrix}
p \\
\dot{p}
\end{pmatrix} +
\begin{pmatrix}
0 \\
\frac{1}{m}
\end{pmatrix}
\begin{pmatrix}
F
\end{pmatrix}
$

Later some changes to the system dynamics will be introduces. They will stay in the same framework of system description.

Please examine the code below that does the following:
- describes the system behaviour in the presence of the control signal
- runs the simulation for a given number of episodes
- visualizes the state and action trajectory, as well as the phase trajectory

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Dyn_block_1D:
    def __init__(self, A, B, x0, v0, dt):
        self.x = np.array([[x0], [v0]])

        self.m = m
        self.dt = dt

        self.A = A

        self.B = B

    def sys_dyn(self, u):
        xdot = self.A @ self.x + self.B @ u

        return xdot

    def integrate(self, xdot):
        self.x += xdot * self.dt

    def get_state(self):
        return self.x

def run_block_simulation(A, B, x0, v0, dt, num_episodes, control_function):
    block = Dyn_block_1D(A, B, x0, v0, dt)

    x_traj = []
    v_traj = []
    u_traj = []

    for i in range(num_episodes):
        x = block.get_state()

        control = control_function(i, x)

        x_traj.append(x[0, 0])
        v_traj.append(x[1, 0])
        u_traj.append(control)

        x_dot = block.sys_dyn(np.array([[control]]))

        block.integrate(x_dot)

    return x_traj, v_traj, u_traj

def run_and_visualize_block(A, B, x0, v0, dt, num_episodes, control_function):
    x_hist, v_hist, u_hist = run_block_simulation(A = A, B = B, x0 = x0, v0 = v0, dt = dt,
                        num_episodes = num_episodes, control_function = control_function)

    plot_1d(x_hist, ylabel = "p")
    plot_1d(v_hist, ylabel = "p_dot")
    plot_1d(u_hist, ylabel = "control_force")

    plt.figure(figsize=(5, 3))
    plt.show()

    plt.figure(figsize=(5, 5))
    plt.plot(x_hist, v_hist)
    plt.xlabel("p")
    plt.ylabel("p_dot")

    plt.show()

    return (x_hist, v_hist, u_hist)

## Running the simulation

There are some plots, that will help us to analyze the behaviour of the system. The first one is the displacement of the mass over time, the second is the velocity, the third is the commanded force.

The fourth plot is the phase trajectory of the system, consisting of the connected dots with each of them representing a state where the system was at a certain moment in time.

Lastly, there is an animation that you are already familiar with.

Let us apply a simple control that will depend purely on time, not on the system state, and observe the results.

In [ ]:
def sin_control(i, x):
    return 20 * np.sin(i / 8.0)

A = np.array([[0.0, 1.0],
              [0.0, 0.0]])

m  = 1.0

B = np.array([[0.0],
              [1.0 / m]])

x0 = 1.0
v0 = 0.0
dt = 0.01

num_episodes = 300
control_function = sin_control

trajectory_sin = run_and_visualize_block(A = A, B = B, x0 = x0, v0 = v0, dt = dt, num_episodes = num_episodes, control_function = control_function)

In [ ]:
animate(trajectory_sin, frames = num_episodes, step=4)

## State-action trajectory cost

Another important part of the control algorithm development is the ability to evaluate the performance of the controller. One of the ways to do so is to measure the approximation of the cost $J$ used in the derivation above. The approximation that will be used is $\hat{J} = \sum_{0}^{N} (x_i^T Q x_i + u_i^T R u_i) \Delta t$.

Please complete the implementation of this function.

In [ ]:
def evaluate_state_action_trajectory(trajectory, Q, R, dt):
    xtr, vtr, utr = trajectory

    cost, cx, cu = 0, 0, 0

    for i in range(len(xtr)):
        xvec = np.array([[xtr[i]],
                         [vtr[i]]])

        uvec = np.array([[utr[i]]])

        # YOUR CODE BELOW
        
        curr_cost = (xvec.T @ Q @ xvec + uvec.T @ R @ uvec) * dt
        
        cx += xvec.T @ Q @ xvec * dt
        cu += uvec.T @ R @ uvec * dt

        # YOUR CODE ABOVE

        cost += curr_cost

    return cost, cx, cu

In [ ]:
Q = np.array([[1, 0],
              [0, 1]])

R = np.eye(1) * 1

traj_cost, cx, cu = evaluate_state_action_trajectory(trajectory_sin, Q, R, dt)

print(traj_cost, cx, cu)

## Proportional-Differential controller

Can you think of a better way to stabilize the system, than the sinosuidal signal, that in reality will not drive it to zero? Please implement a PD negative feedback in the form of $F = - K_p p - K_d \dot{p}$. Compare the cost with the one that the sinosuidal controller ended up with.

In [ ]:
def PD_control(i, x):
    # YOUR CODE BELOW
    u = - 3 * x[0, 0] - 1 * x[1, 0]
    # YOUR CODE ABOVE

    return u

A = np.array([[0.0, 1.0],
              [0.0, 0.0]])

m = 1.0

B = np.array([[0.0],
              [1.0 / m]])

x0 = 1.0
v0 = 0.3
dt = 0.01

num_episodes = 1300
control_function = PD_control

trajectory_PD = run_and_visualize_block(A = A, B = B, x0 = x0, v0 = v0, dt = dt, num_episodes = num_episodes, control_function = control_function)

In [ ]:
animate(trajectory_PD, frames = num_episodes)

In [ ]:
Q = np.array([[1, 0],
              [0, 1]])

R = np.eye(1) * 1

traj_cost, cx, cu = evaluate_state_action_trajectory(trajectory_PD, Q, R, dt)

print(traj_cost, cx, cu)

## Linear Quadratic Regulator

Let us recall that LQR feedback has the form of $u = - R^{-1} B^T P x$, where $P$ is the solution of an Algebraic Riccati Equation $A^T P + P A + Q - P B R^{-1} B^T P = 0_{n \times n}$.

For the given system

$A =
 \begin{pmatrix}
0 & 1 \\
0 & 0
\end{pmatrix}$

$B =
\begin{pmatrix}
0 \\
\frac{1}{m}
\end{pmatrix}
$

And P could be presented in the form of

$P =
\begin{pmatrix}
p_1 & p_2 \\
p_2 & p_3
\end{pmatrix}$

taking into account its property of being symmetric.

In the explicit form ARE reads

$\begin{pmatrix}
0 & 0 \\
1 & 0
\end{pmatrix}
\begin{pmatrix}
p_1 & p_2 \\
p_2 & p_3
\end{pmatrix} +
\begin{pmatrix}
p_1 & p_2 \\
p_2 & p_3
\end{pmatrix}
\begin{pmatrix}
0 & 1 \\
0 & 0
\end{pmatrix} + Q +
\begin{pmatrix}
p_1 & p_2 \\
p_2 & p_3
\end{pmatrix}
\begin{pmatrix}
0 \\
\frac{1}{m}
\end{pmatrix}
R^{-1}
\begin{pmatrix}
0 & \frac{1}{m}
\end{pmatrix}
\begin{pmatrix}
p_1 & p_2 \\
p_2 & p_3
\end{pmatrix} =
\begin{pmatrix}
0 & 0\\
0 & 0
\end{pmatrix}
$

Voluntarely assuming the mass to be equal to one and setting $Q$ and $R$ to be identity matrices of proper dimensions (exactly as they appear in the code), we can reformulate this equation as

$
\begin{pmatrix}
1 - p_2^2 & p_1 - p_2 p_3\\
p_1 - p_2 p_3 & 2 p_2 - p_3^2 + 1
\end{pmatrix} =
\begin{pmatrix}
0 & 0\\
0 & 0
\end{pmatrix}
$

Getting rid of the solutions that stabilize the system at negative time, we finally get

$P =
\begin{pmatrix}
\sqrt{3} & 1 \\
1 & \sqrt{3}
\end{pmatrix}$

Which results in the feedback gain matrix $K = R^{-1} B^T P = \begin{pmatrix} 1 & \sqrt{3} \end{pmatrix}$ and the feedback law $u = - p - \sqrt{3} \dot{p}$.

Moreover, it is now possible to calculate the exact value of the cost function (see the end on the controller derivation), because it is formulated in terms of the $P$ matrix.

For the given initial state of $x_0 = \begin{pmatrix}
1 \\
0.3
\end{pmatrix}$

the cost is equal to $x_0^T P x_0 \approx 2.49$.

In [ ]:
Q = np.array([[1, 0],
              [0, 1]])

R = np.eye(1) * 1

A = np.array([[0.0, 1.0],
              [0.0, 0.0]])

P = np.array([[math.sqrt(3), 1],
              [1, math.sqrt(3)]])

m  = 1.0

B = np.array([[0],
              [1 / m]])

def LQR_control(i, x):
    # YOUR CODE BELOW
    
    K = np.linalg.inv(R) @ B.T @ P

    # YOUR CODE ABOVE

    u = - K @ x

    #print(u)

    return u[0, 0]

x0 = 1.0
v0 = 0.3
dt = 0.01

num_episodes = 1300
control_function = LQR_control

trajectory = run_and_visualize_block(A = A, B = B, x0 = x0, v0 = v0, dt = dt, num_episodes = num_episodes, control_function = control_function)

In [ ]:
animate(trajectory, frames = num_episodes)

Let us evaluate the performance of this controller. Why is the exact value of the cost different from the theoretically obtained one?

In [ ]:
Q = np.array([[1, 0],
              [0, 1]])

R = np.eye(1) * 1

traj_cost = evaluate_state_action_trajectory(trajectory, Q, R, dt)

print(traj_cost)

## Solving ARE with software tools

In [ ]:
import numpy as np
from scipy import linalg

A = np.array([[0, 1],
              [0, 0]])

B = np.array([[0],
              [1]])

Q = np.array([[1, 0],
              [0, 1]])

R = np.array([[1]])

P = linalg.solve_continuous_are(A, B, Q, R)

print(P)

np.allclose(A.T @ P + P @ A - P @ B @ np.linalg.inv(R) @ B.T @ P, -Q)

## The impact of the parameters on the cost and on the state-action trajectory

Now let us perform a series of experiments with varying the parameters and the values in the $Q$ and $R$ matrices. But first, let us wrap the whole simulation in a single function for convenience.

In [ ]:
def run_visualize_evaluate_LQR_controller(A, B, Q_multiplier, R_multiplier, num_episodes,
                                          dt, x0, v0):
    Q = np.array([[1, 0],
                  [0, 1]]) * Q_multiplier

    R = np.eye(1) * R_multiplier

    P = linalg.solve_continuous_are(A, B, Q, R)

    # INSERT YOUR CODE FROM ABOVE HERE

    # INSERT YOUR CODE FROM ABOVE HERE

    print(K)

    def LQR_control(i, x):
        K = np.linalg.inv(R) @ B.T @ P

        u = - K @ x

        return u[0, 0]

    control_function = LQR_control

    trajectory = run_and_visualize_block(A = A, B = B, x0 = x0, v0 = v0, dt = dt,
                      num_episodes = num_episodes, control_function = control_function)

    traj_cost = evaluate_state_action_trajectory(trajectory, Q, R, dt)

    print(traj_cost)

In [ ]:
A = np.array([[0.0, 1.0],
              [0.0, 0.0]])

m  = 1.0

B = np.array([[0],
              [1 / m]])

num_episodes = 3000

x0 = 1.0
v0 = 0.3

dt = 0.01

run_visualize_evaluate_LQR_controller(A = A, B = B, Q_multiplier = 1, R_multiplier = 100,
                                      num_episodes = num_episodes, dt = dt, x0 = x0, v0 = v0)

Try changing the simulation parameters and answer the following questions:
- Does the increase of the number of the simulation steps from $300$ to $1300$ change the total cost? Why so?
- Does the increase of the number of the simulation steps from $1300$ to $13000$ change the total cost? Why so?

Now please set the number of simulation steps to $3000$.

- How does the change of the time step $\Delta t$ from $0.01$ to $0.001$ along with a proper increase in the number of the simulation steps (in order to conserve the total duration of the episode) change the total cost? How exactly?
- Try setting R_multiplier to $0.1$. What happened to the system trajectory? What happened to the gain matrix?

## Spring-mass system

Now let us connect the mass to the wall with a spring of stifness $k$. This will change the $A$ matrix to the following:

$A =
\begin{pmatrix}
0 & 1 \\
-\frac{k}{m} & 0
\end{pmatrix}$

Try different values of $Q$ and $R$ matrices:
- Default values of $1$ and $1$
- With very expensive control
- With very cheap control

Examine the results and describe them.

In [ ]:
m  = 1.0
k = 1.0

A = np.array([[0.0, 1.0],
              [- k / m, 0.0]])

B = np.array([[0],
              [1 / m]])

num_episodes = 3000

x0 = 1.0
v0 = 0.3

dt = 0.01

run_visualize_evaluate_LQR_controller(A = A, B = B, Q_multiplier = 1, R_multiplier = 0.100,
                                      num_episodes = num_episodes, dt = dt, x0 = x0, v0 = v0)

### Home assignment

1) apply LQR to the problem of stabilization of the system of two pendulums from the previous assignment. Does the optimal cost match with the best one you have obtained previously? Why?
2) try decreasing $R$ matrix by a factor of 10. Did the system behaviour change? Why?
3) apply LQR to the same system, but consisting of two inverted pendulums. Is the behaviour if the system generally same as for normal pendulums or not? Why?
4) change $R$ matrix back and run the simulation. Did it qualitatively influence the behaviour if the system? Why?

For each question a separate subsection should be created with the simulation plots and **elaborate answers** to the questions.

1) примените LQR к задаче стабилизации системы из двух маятников из прошлой домашки. Совпадает ли стоимость оптимальной траектории с той, которая получалась без LQR? Почему?
2) уменьшите значение матрицы $R$ в 10 раз. Как это повлияло на поведение системы?
3) примените LQR к системе из двух обратных маятников, связанных пружиной. Похоже ли поведение системы на поведение системы из обычных маятников? Почему?
4) увеличьте $R$ обратно и запустите симуляцию снова. Повлияло ли это качественно на поведение системы?

Каждой подзадачи создайте пожалуйста отдельную подсекцию (в маркдауне можно писать заголовки с помощью значка решетки) с графиками поведения систем и подробными текстовыми ответами на вопросы.